In [17]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation

In [6]:
df = pd.read_csv("../Data/imu_data_sample.csv", index_col="index")

In [9]:
initial_q = df.loc[1].to_list()
final_q = df.loc[3].to_list()

In [11]:
q_1 = np.array(initial_q)
q_2 = np.array(final_q)

In [15]:
delta_q = q_1 * np.conjugate(q_2)
print(delta_q)

[0.58724517 0.0571773  0.00981005 0.34576747]


In [16]:
delta_q /= np.linalg.norm(delta_q)
delta_q

array([0.85861759, 0.08359956, 0.01434339, 0.5055504 ])

In [19]:
delta_rotation = Rotation.from_quat(delta_q)
delta_rotation

In [21]:
ang_vel = delta_rotation.as_rotvec() / 0.1
print(ang_vel)

[20.71469843  2.0168928   0.34604338]


In [36]:
TIME_INTERVAL = 0.1

def get_angular_vel(list1, list2):
    q_1 = np.array(list1)
    q_2 = np.array(list2)

    delta_q = q_1 * np.conjugate(q_2)
    delta_q /= np.linalg.norm(delta_q)
    delta_rotation = Rotation.from_quat(delta_q)

    rotation = delta_rotation.as_rotvec() / TIME_INTERVAL

    return rotation.tolist()

In [30]:
angular_vel_list = []

for index in range(len(df)-1):
    list1 = df.loc[index].to_list()
    list2 = df.loc[index+1].to_list()
    angular_vel_list.append(get_angular_vel(list1, list2))

In [32]:
maximo = 0
minimo = 0
for rotation in angular_vel_list:
    for val in rotation:
        if (maximo < val):
            maximo = val
        if (minimo > val):
            minimo = val
print(maximo, minimo)

24.378949978459552 -0.39973262576280855


In [33]:
#according to google, mavic 2 pro can handle 3.5 rad/sec of angular velocity so I'll go with 7 because is a lucky number
MAX_ROTATION = 7
MIN_ROTATION = -7
def min_max_scaler(data, min_value, max_value):
    scaled_data = (data - min_value) / (max_value - min_value)
    if scaled_data > 1:
        scaled_data = 1
    elif scaled_data < 0:
        scaled_data = 0
    return  scaled_data

In [35]:
angular_vel_list[0]

def rotation_scaler(rotation):
    for i in rotation:
        rotation[rotation.index(i)] = min_max_scaler(i, MIN_ROTATION, MAX_ROTATION)
    return rotation

rotation_scaler(angular_vel_list[0])

[1, 0.6440637712690125, 0.5247173841791374]

In [44]:
def get_angular_accel(list1, list2, list3):
    ang_vel1 = get_angular_vel(list1, list2)
    ang_vel2 = get_angular_vel(list2, list3)
    angular_acceleration  = []
    for val in range(len(ang_vel1)):
        angular_acceleration.append((ang_vel2[val] - ang_vel1[val])/TIME_INTERVAL)
    return angular_acceleration

In [45]:
angular_accel_list = []

for index in range(len(df)-2):
    list1 = df.loc[index].to_list()
    list2 = df.loc[index+1].to_list()
    list3 = df.loc[index+2].to_list()
    angular_accel_list.append(get_angular_accel(list1, list2, list3))

maximo = 0
minimo = 0
for rotation in angular_accel_list:
    for val in rotation:
        if (maximo < val):
            maximo = val
        if (minimo > val):
            minimo = val
print(maximo, minimo)

36.80826077786122 -21.942560791231788


In [58]:
MAX_ACCELERATION = MAX_ROTATION**2
MIN_ACCELERATION = - MIN_ROTATION**2

def angular_velocity(data1, data2):
    velocity = get_angular_vel(data1, data2)
    return rotation_scaler(velocity)

def angular_acceleration(data1, data2, data3):
    acceleration = get_angular_accel(data1, data2, data3)
    return rotation_scaler(acceleration)

In [59]:
import timeit

data1 = df.loc[0].to_list()
data2 = df.loc[1].to_list()
data3 = df.loc[2].to_list()

vel_time = timeit.timeit((lambda: angular_velocity(data1, data2)) ,number=1)
accel_time = timeit.timeit((lambda : angular_acceleration(data1, data2, data3)), number=1)

print(vel_time, accel_time)

9.920000047713984e-05 6.880000000819564e-05
